# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark
findspark.init('/home/kajili/spark-2.4.5-bin-hadoop2.7')
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('logreg-consult').getOrCreate()

In [2]:
rawData = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [3]:
rawData.show(5)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|  Cynthia Norton|37.0|    

In [4]:
rawData.head()

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)

In [5]:
rawData.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
rawData.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
relevant_cols = rawData.select(['Age',
                                'Total_Purchase',
                                'Account_Manager',
                                'Years',
                                'Num_Sites',
                                'Churn'
                                ])

In [8]:
relevant_cols.show(5)

+----+--------------+---------------+-----+---------+-----+
| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+----+--------------+---------------+-----+---------+-----+
|42.0|       11066.8|              0| 7.22|      8.0|    1|
|41.0|      11916.22|              0|  6.5|     11.0|    1|
|38.0|      12884.75|              0| 6.67|     12.0|    1|
|42.0|       8010.76|              0| 6.71|     10.0|    1|
|37.0|       9191.58|              0| 5.56|      9.0|    1|
+----+--------------+---------------+-----+---------+-----+
only showing top 5 rows



In [9]:
final_data = relevant_cols.na.drop()

In [10]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoderEstimator,StringIndexer)

In [11]:
# location_indexer = StringIndexer(inputCol='Location',outputCol='LocationIndex')
# location_encoder = OneHotEncoderEstimator(inputCols=['LocationIndex'],outputCols=['LocationVec'])

In [12]:
# company_indexer = StringIndexer(inputCol='Company',outputCol='CompanyIndex')
# company_encoder = OneHotEncoderEstimator(inputCols=['CompanyIndex'],outputCols=['CompanyVec'])

In [13]:
final_data.columns

['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites', 'Churn']

In [14]:
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'],
                            outputCol='features')

In [15]:
from pyspark.ml.classification import LogisticRegression

In [16]:
from pyspark.ml import Pipeline

In [17]:
log_reg_customer_obj = LogisticRegression(featuresCol='features', labelCol='Churn')

In [18]:
pipeline = Pipeline(stages=[assembler,log_reg_customer_obj])
                            #location_indexer,company_indexer,
                            #location_encoder,company_encoder,
                           

In [19]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [20]:
customer_churn_model = pipeline.fit(train_data)

In [21]:
# customer_churn_model.evaluate(test_data)

In [22]:
results = customer_churn_model.transform(test_data)
type(results)

pyspark.sql.dataframe.DataFrame

In [23]:
results.select('Churn', 'prediction').show(100)

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|     

In [24]:
results.select('Churn', 'prediction').describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                285|                285|
|   mean|0.17543859649122806|0.15087719298245614|
| stddev| 0.3810108412907983| 0.3585587381003972|
|    min|                  0|                0.0|
|    max|                  1|                1.0|
+-------+-------------------+-------------------+



In [29]:
results.show()

+----+--------------+---------------+-----+---------+-----+--------------------+--------------------+--------------------+----------+
| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|            features|       rawPrediction|         probability|prediction|
+----+--------------+---------------+-----+---------+-----+--------------------+--------------------+--------------------+----------+
|28.0|      11245.38|              0| 6.72|      8.0|    0|[28.0,11245.38,0....|[4.61300168030154...|[0.99017548780141...|       0.0|
|29.0|       8688.17|              1|  5.7|      9.0|    1|[29.0,8688.17,1.0...|[3.08220266060859...|[0.95615262351480...|       0.0|
|29.0|       9617.59|              0| 5.49|      8.0|    0|[29.0,9617.59,0.0...|[5.05827861987195...|[0.99368365773952...|       0.0|
|29.0|      11274.46|              1| 4.43|      8.0|    0|[29.0,11274.46,1....|[4.93997730897524...|[0.99289606622698...|       0.0|
|29.0|      12711.15|              0| 5.74|      7.0|    0|[29

In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [26]:
my_eval_obj = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                            labelCol='Churn')

In [27]:
AreaUnderCurve = my_eval_obj.evaluate(results)

In [28]:
AreaUnderCurve

0.7965957446808511